In [30]:
import torch
import tiktoken


In [94]:
text = "Hello, my name is Jaafar and I love NLP" #, because it allows us to capture syntactic and semantic from natural language."


enc = tiktoken.get_encoding("cl100k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"
# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4")

# here are all the unique characters that occur in this text
chars = sorted(list(set(text.split(" "))))
vocab_size = len(chars)

# here are all the unique characters that occur in this text
unique_words = sorted(list(set(text)))

tokens_text = enc.encode(text)

vocab_size = len(tokens_text)


jaafar_tokens = {i:token for i, token in enumerate(tokens_text)}
jaafar_tokens_reversed = {token:i for i, token in enumerate(tokens_text)}


encode = lambda s: [jaafar_tokens_reversed[c] for c in enc.encode(s)] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([enc.decode([jaafar_tokens[i]]) for i in l]) # decoder: take a list of integers, output a string


example = "Hello, my name is Jaafar"
assert decode(encode(example)) == example

In [95]:
jaafar_tokens

{0: 9906,
 1: 11,
 2: 856,
 3: 836,
 4: 374,
 5: 23720,
 6: 2642,
 7: 277,
 8: 323,
 9: 358,
 10: 3021,
 11: 452,
 12: 12852}

In [96]:
jaafar_tokens_reversed

{9906: 0,
 11: 1,
 856: 2,
 836: 3,
 374: 4,
 23720: 5,
 2642: 6,
 277: 7,
 323: 8,
 358: 9,
 3021: 10,
 452: 11,
 12852: 12}

In [97]:
print(text)
print(encode(text))
print(len(text.split()))
print(len(tokens_text))


Hello, my name is Jaafar and I love NLP
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
9
13


In [98]:
data = torch.tensor(encode(text), dtype=torch.long)
data

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [99]:
device = "cpu"
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    
    # TODO : draw random number from 0 to [len(data)-block_size], in just one dimension. And we want N=batch_size element
    # We want to select randomly N=batch_size times a block size ! 
    ix = torch.randint(len(data) - block_size, (batch_size,))

    # TODO : We draw from this random generate our block size = Context size.
    x = torch.stack([data[i:i+block_size] for i in ix])

    # we take the next block size as a prediction
    # We just add one so that for every index, the prediction is at the same row in the same colum 
    # from X context 0 - N in a given row X  => prediction colum N+1 in y
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    x, y = x.to(device), y.to(device)
    return x, y

In [123]:
batch_size = 4 # How many independent sequence we will process in parallel ?
block_size = 5 # How is the maximum context length N

train_data = data
X, y = get_batch("train")

In [124]:
data = train_data
a = len(data) - block_size
b = (batch_size,)
print("data = ",data)
print("block_size = ", block_size)
print("batch_size = ", batch_size)
print(a,b)
torch.randint(a, b)

data =  tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
block_size =  5
batch_size =  4
8 (4,)


tensor([3, 5, 5, 0])

In [125]:
data

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [126]:
[data[i:i+block_size] for i in [1, 1, 1, 4]]

[tensor([1, 2, 3, 4, 5]),
 tensor([1, 2, 3, 4, 5]),
 tensor([1, 2, 3, 4, 5]),
 tensor([4, 5, 6, 7, 8])]

In [127]:
[data[i+1:i+block_size+1] for i in [1, 1, 1, 4]]

[tensor([2, 3, 4, 5, 6]),
 tensor([2, 3, 4, 5, 6]),
 tensor([2, 3, 4, 5, 6]),
 tensor([5, 6, 7, 8, 9])]

In [128]:
help(torch.randint)

Help on built-in function randint in module torch:

randint(...)
    randint(low=0, high, size, \*, generator=None, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False) -> Tensor
    
    Returns a tensor filled with random integers generated uniformly
    between :attr:`low` (inclusive) and :attr:`high` (exclusive).
    
    The shape of the tensor is defined by the variable argument :attr:`size`.
    
    .. note::
        With the global dtype default (``torch.float32``), this function returns
        a tensor with dtype ``torch.int64``.
    
    Args:
        low (int, optional): Lowest integer to be drawn from the distribution. Default: 0.
        high (int): One above the highest integer to be drawn from the distribution.
        size (tuple): a tuple defining the shape of the output tensor.
    
    Keyword args:
        generator (:class:`torch.Generator`, optional): a pseudorandom number generator for sampling
        out (Tensor, optional): the output

In [129]:
torch.randint(block_size, (batch_size,))
# ()

tensor([0, 1, 0, 4])

In [130]:
X, y = get_batch("train")

print(X) 

tensor([[ 4,  5,  6,  7,  8],
        [ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10],
        [ 2,  3,  4,  5,  6]])


In [131]:
print(y)

tensor([[ 5,  6,  7,  8,  9],
        [ 2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11],
        [ 3,  4,  5,  6,  7]])


In [132]:
for i in range(len(X)):
    for j in range(len(row)):
        
        print("X = ", X[i,:j+1], " ==> y = ", y[i,j])



X =  tensor([4])  ==> y =  tensor(5)
X =  tensor([4, 5])  ==> y =  tensor(6)
X =  tensor([4, 5, 6])  ==> y =  tensor(7)
X =  tensor([4, 5, 6, 7])  ==> y =  tensor(8)
X =  tensor([4, 5, 6, 7, 8])  ==> y =  tensor(9)
X =  tensor([1])  ==> y =  tensor(2)
X =  tensor([1, 2])  ==> y =  tensor(3)
X =  tensor([1, 2, 3])  ==> y =  tensor(4)
X =  tensor([1, 2, 3, 4])  ==> y =  tensor(5)
X =  tensor([1, 2, 3, 4, 5])  ==> y =  tensor(6)
X =  tensor([6])  ==> y =  tensor(7)
X =  tensor([6, 7])  ==> y =  tensor(8)
X =  tensor([6, 7, 8])  ==> y =  tensor(9)
X =  tensor([6, 7, 8, 9])  ==> y =  tensor(10)
X =  tensor([ 6,  7,  8,  9, 10])  ==> y =  tensor(11)
X =  tensor([2])  ==> y =  tensor(3)
X =  tensor([2, 3])  ==> y =  tensor(4)
X =  tensor([2, 3, 4])  ==> y =  tensor(5)
X =  tensor([2, 3, 4, 5])  ==> y =  tensor(6)
X =  tensor([2, 3, 4, 5, 6])  ==> y =  tensor(7)


In [133]:
vocab_size

13

In [134]:
X

tensor([[ 4,  5,  6,  7,  8],
        [ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10],
        [ 2,  3,  4,  5,  6]])

In [135]:
X.shape # row, colums ! BATCH, CONTEXT (time)

torch.Size([4, 5])

In [236]:
import torch 
import torch.nn as nn
from torch.nn import functional as F 
torch.manual_seed(69)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token will be projected into a 13*13 dimension
        # Given a context of words, what is its meaning in our world (vocab dimension)
        # what is also happening befin the scene is that each token is encode into a one hot encoder before feedforward
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, x, targets):

        # row, colums ! BATCH, CONTEXT (time)
        logits = self.token_embedding_table(x) # BATCH, CONTEXT, VOCAB_SIZE = Channel !

        loss = F.cross_entropy(logits, targets) # it want a B,T,C instead of a B,C,T
        
        return logits, loss

In [237]:
X,y

(tensor([[ 4,  5,  6,  7,  8],
         [ 1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10],
         [ 2,  3,  4,  5,  6]]),
 tensor([[ 5,  6,  7,  8,  9],
         [ 2,  3,  4,  5,  6],
         [ 7,  8,  9, 10, 11],
         [ 3,  4,  5,  6,  7]]))

In [239]:
y.shape

torch.Size([4, 5])

In [238]:
m = BigramLanguageModel(vocab_size=vocab_size)

logits, loss = m(X,y)

logits.shape

RuntimeError: Expected target size [4, 13], got [4, 5]

In [202]:
logits

tensor([[[ 0.8756,  0.2960,  0.6394,  1.2406, -1.2100, -0.9481,  0.6444,
           0.2188, -1.2224, -0.9322, -0.3832,  1.4027,  0.4913],
         [ 1.0152, -0.0184, -0.8487,  1.8939, -0.0325,  0.3191, -0.5700,
          -0.4292, -1.0216, -0.5285,  1.6272, -0.2411, -2.1021],
         [-0.0179,  0.2241,  0.4389,  1.6109, -0.7284, -1.6016,  0.4660,
          -0.1961,  0.3063, -1.8420, -0.4396,  1.3367,  0.3671],
         [ 1.3794,  1.1527,  0.0762,  0.0557, -0.6163, -1.2772,  0.4905,
          -0.4098,  0.0517,  0.7329, -0.7448,  2.2900,  0.1214],
         [-1.3227,  1.0725, -1.6382,  1.5207,  0.5975,  0.5968, -0.5111,
          -0.2617, -0.4817,  0.4692,  0.4951, -0.2140, -1.0078]],

        [[ 0.3527, -0.0916,  0.0341, -0.8986,  0.1022, -0.6627, -0.1350,
          -0.3983, -1.7892,  1.2785,  1.3351, -0.3066,  1.0382],
         [ 1.2762,  0.0419, -1.2794, -1.8432,  0.8633, -1.7786, -0.8080,
          -0.8735,  0.9367, -1.2319,  1.5287, -0.2759, -0.8625],
         [-0.1915, -0.4807, -1.4

In [215]:
logits.shape

torch.Size([4, 5, 13])

In [231]:
exp = logits.exp()

probs = exp/torch.sum(exp, dim=1, keepdim=True)

probs.sum(2)

#probs.argmax(2, keepdim=False)


tensor([[2.4754, 2.4755, 2.3665, 3.1145, 2.5681],
        [2.6159, 2.4354, 1.9926, 3.2199, 2.7362],
        [2.5672, 2.9067, 2.2769, 2.4989, 2.7504],
        [2.4330, 2.2624, 2.9257, 2.7202, 2.6587]], grad_fn=<SumBackward1>)

In [235]:
torch.softmax(logits,1,dtype=torch.float32).sum(2)

tensor([[2.4754, 2.4755, 2.3665, 3.1145, 2.5681],
        [2.6159, 2.4354, 1.9926, 3.2199, 2.7362],
        [2.5672, 2.9067, 2.2769, 2.4989, 2.7504],
        [2.4330, 2.2624, 2.9257, 2.7202, 2.6587]], grad_fn=<SumBackward1>)

In [223]:
X

tensor([[ 4,  5,  6,  7,  8],
        [ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10],
        [ 2,  3,  4,  5,  6]])

In [204]:
embedding_weights = m.token_embedding_table.weight
print(embedding_weights.shape)
embedding_weights

torch.Size([13, 13])


Parameter containing:
tensor([[-0.5300, -1.3035,  0.4438,  1.2221,  1.0395,  0.9608,  0.4214,  0.7452,
         -1.8389, -1.2497, -0.2485,  0.1428, -1.0509],
        [ 0.3527, -0.0916,  0.0341, -0.8986,  0.1022, -0.6627, -0.1350, -0.3983,
         -1.7892,  1.2785,  1.3351, -0.3066,  1.0382],
        [ 1.2762,  0.0419, -1.2794, -1.8432,  0.8633, -1.7786, -0.8080, -0.8735,
          0.9367, -1.2319,  1.5287, -0.2759, -0.8625],
        [-0.1915, -0.4807, -1.4154,  0.0934, -0.2420, -1.0300, -0.2034, -0.6882,
         -0.0178,  1.1983, -0.8180, -0.7297,  0.8256],
        [ 0.8756,  0.2960,  0.6394,  1.2406, -1.2100, -0.9481,  0.6444,  0.2188,
         -1.2224, -0.9322, -0.3832,  1.4027,  0.4913],
        [ 1.0152, -0.0184, -0.8487,  1.8939, -0.0325,  0.3191, -0.5700, -0.4292,
         -1.0216, -0.5285,  1.6272, -0.2411, -2.1021],
        [-0.0179,  0.2241,  0.4389,  1.6109, -0.7284, -1.6016,  0.4660, -0.1961,
          0.3063, -1.8420, -0.4396,  1.3367,  0.3671],
        [ 1.3794,  1.1527,

In [183]:
text

'Hello, my name is Jaafar and I love NLP'

In [184]:
X

tensor([[ 4,  5,  6,  7,  8],
        [ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10],
        [ 2,  3,  4,  5,  6]])

In [162]:
out

tensor([[[ 0.8756,  0.2960,  0.6394,  1.2406, -1.2100, -0.9481,  0.6444,
           0.2188, -1.2224, -0.9322, -0.3832,  1.4027,  0.4913],
         [ 1.0152, -0.0184, -0.8487,  1.8939, -0.0325,  0.3191, -0.5700,
          -0.4292, -1.0216, -0.5285,  1.6272, -0.2411, -2.1021],
         [-0.0179,  0.2241,  0.4389,  1.6109, -0.7284, -1.6016,  0.4660,
          -0.1961,  0.3063, -1.8420, -0.4396,  1.3367,  0.3671],
         [ 1.3794,  1.1527,  0.0762,  0.0557, -0.6163, -1.2772,  0.4905,
          -0.4098,  0.0517,  0.7329, -0.7448,  2.2900,  0.1214],
         [-1.3227,  1.0725, -1.6382,  1.5207,  0.5975,  0.5968, -0.5111,
          -0.2617, -0.4817,  0.4692,  0.4951, -0.2140, -1.0078]],

        [[ 0.3527, -0.0916,  0.0341, -0.8986,  0.1022, -0.6627, -0.1350,
          -0.3983, -1.7892,  1.2785,  1.3351, -0.3066,  1.0382],
         [ 1.2762,  0.0419, -1.2794, -1.8432,  0.8633, -1.7786, -0.8080,
          -0.8735,  0.9367, -1.2319,  1.5287, -0.2759, -0.8625],
         [-0.1915, -0.4807, -1.4

In [147]:
torch.sum(out, dim=2, keepdim=False)

tensor([[ 1.1127, -0.9366, -0.0756,  3.3023, -0.6855],
        [-0.1412, -4.3060, -3.6993,  1.1127, -0.9366],
        [-0.0756,  3.3023, -0.6855, -0.1063,  2.1730],
        [-4.3060, -3.6993,  1.1127, -0.9366, -0.0756]], grad_fn=<SumBackward1>)